In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import pyarrow.parquet as pq

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
train_events = pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')
# drop NAN rows
train_events = train_events.dropna(axis=0)
train_events

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400
...,...,...,...,...,...
14501,fe90110788d2,32,wakeup,547152.0,2017-09-05T09:26:00-0400
14502,fe90110788d2,33,onset,556560.0,2017-09-05T22:30:00-0400
14503,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400
14504,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400


In [7]:
series_id = train_events.loc[0]['series_id']
series_id

'038441c925bb'

train_series, test_series -> 5 min intervals, no NAN

In [11]:
parquet_train_series = '/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet'
#train_series = pq.read_table(parquet_train_series).to_pandas()

train_series = pq.read_table(parquet_train_series,
                             filters=[[('series_id', '=', series_id)],]).to_pandas()
train_series

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
389875,038441c925bb,389875,2018-09-06T04:59:35-0400,-27.373899,0.0110
389876,038441c925bb,389876,2018-09-06T04:59:40-0400,-27.493799,0.0110
389877,038441c925bb,389877,2018-09-06T04:59:45-0400,-27.533701,0.0111
389878,038441c925bb,389878,2018-09-06T04:59:50-0400,-28.003599,0.0111


In [22]:
def series_max(series):
    return series.max()

In [24]:
def series_min(series):
    return series.min()

In [23]:
#train_series_max = train_series.max()
train_series_max = series_max(train_series)
train_series_max

series_id                038441c925bb
step                           389879
timestamp    2018-09-06T04:59:55-0400
anglez                      89.046898
enmo                           3.3255
dtype: object

In [25]:
#train_series_min = train_series.min()
train_series_min = series_min(train_series)
train_series_min

series_id                038441c925bb
step                                0
timestamp    2018-08-14T15:30:00-0400
anglez                     -89.576797
enmo                              0.0
dtype: object

In [8]:
parquet_test_series = '/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet'
# test_series = pq.read_table(parquet_test_series).to_pandas()
test_series = pq.read_table(parquet_test_series,
                             filters=[[('series_id', '=', series_id)],]).to_pandas()
test_series

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
145,038441c925bb,145,2018-08-14T15:42:05-0400,-79.990601,0.0135
146,038441c925bb,146,2018-08-14T15:42:10-0400,-80.028999,0.0136
147,038441c925bb,147,2018-08-14T15:42:15-0400,-80.009300,0.0137
148,038441c925bb,148,2018-08-14T15:42:20-0400,-80.066902,0.0135


In [26]:
test_series_max = series_max(test_series)
test_series_max

series_id                038441c925bb
step                              149
timestamp    2018-08-14T15:42:25-0400
anglez                      68.460503
enmo                           0.0691
dtype: object

In [27]:
test_series_min = series_min(test_series)
test_series_min

series_id                038441c925bb
step                                0
timestamp    2018-08-14T15:30:00-0400
anglez                     -80.233101
enmo                            0.007
dtype: object

In [14]:
train_series = train_series.dropna(axis=0)
test_series = test_series.dropna(axis=0)

In [15]:
display(train_series)
display(test_series)

,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
389875,038441c925bb,389875,2018-09-06T04:59:35-0400,-27.373899,0.0110
389876,038441c925bb,389876,2018-09-06T04:59:40-0400,-27.493799,0.0110
389877,038441c925bb,389877,2018-09-06T04:59:45-0400,-27.533701,0.0111
389878,038441c925bb,389878,2018-09-06T04:59:50-0400,-28.003599,0.0111


,series_id,step,timestamp,anglez,enmo
0,038441c925bb,0,2018-08-14T15:30:00-0400,2.636700,0.0217
1,038441c925bb,1,2018-08-14T15:30:05-0400,2.636800,0.0215
2,038441c925bb,2,2018-08-14T15:30:10-0400,2.637000,0.0216
3,038441c925bb,3,2018-08-14T15:30:15-0400,2.636800,0.0213
4,038441c925bb,4,2018-08-14T15:30:20-0400,2.636800,0.0215
...,...,...,...,...,...
145,038441c925bb,145,2018-08-14T15:42:05-0400,-79.990601,0.0135
146,038441c925bb,146,2018-08-14T15:42:10-0400,-80.028999,0.0136
147,038441c925bb,147,2018-08-14T15:42:15-0400,-80.009300,0.0137
148,038441c925bb,148,2018-08-14T15:42:20-0400,-80.066902,0.0135
